In [ ]:
# Third-party
import astropy.coordinates as coord
from astropy.table import Table
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy.stats import binned_statistic
import gala.coordinates as gc
from pyia import GaiaData

In [ ]:
gd1 = GaiaData('/Users/adrian/projects/gd1-dr2/data/gd1-master.fits')

In [ ]:
iso = Table.read('/Users/adrian/data/Isochrones/PARSEC/FeH_-1.4_iso.fits')
iso_mask = (iso['stage']>=0) & (iso['stage']<3) & (iso['log(age/yr)'] == 10)
iso = iso[iso_mask]

iso_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value

## What is the linear mass density of stars along GD-1?

In [ ]:
stream = gd1[gd1.pm_mask & gd1.gi_cmd_mask]

segment_mask = (stream.phi1 > -40*u.deg) & (stream.phi1 < -30*u.deg)
main = stream[segment_mask & (np.abs(stream.phi2) < 0.5*u.deg)]
spur = stream[segment_mask & (stream.phi2 > 0.5*u.deg) & (stream.phi2 < 2.*u.deg)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.plot(stream.phi1, stream.phi2,
        marker='.', color='k', alpha=0.5, ls='none')

ax.set_xlim(-75, -10)
ax.set_ylim(-10, 10)

ax.set_xlabel(r'$\phi_1$ [deg]')
ax.set_ylabel(r'$\phi_2$ [deg]')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))

ax.plot(stream.phi1, stream.phi2,
        marker='.', color='#aaaaaa', alpha=0.5, ls='none')

ax.plot(main.phi1, main.phi2,
        marker='.', color='k', alpha=0.8, ls='none')

ax.plot(spur.phi1, spur.phi2,
        marker='.', color='tab:red', alpha=0.8, ls='none')

ax.set_xlim(-55, -25)
ax.set_ylim(-10, 10)

ax.set_xlabel(r'$\phi_1$ [deg]')
ax.set_ylabel(r'$\phi_2$ [deg]')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

ax.plot(main.g0 - main.i0, main.g0,
        marker='.', ls='none', alpha=0.5)
ax.set_xlim(-1, 2)
ax.set_ylim(21.5, 14)
fig.tight_layout()

In [ ]:
gd1_lf_mask = (iso_g > 18) & (iso_g < 21)
gd1_lf_iso = iso[gd1_lf_mask]

nstream = ((main.g0 > 18*u.mag) & (main.g0 < 21*u.mag)).sum()
nspur = ((spur.g0 > 18*u.mag) & (spur.g0 < 21*u.mag)).sum()
nstream, nspur

In [ ]:
fac = np.sum((gd1_lf_iso['int_IMF'][1:] - gd1_lf_iso['int_IMF'][:-1]) / u.Msun)
M_stream = nstream / fac
M_spur = nspur / fac

stream_linear_dens = M_stream / (10*u.deg)
spur_linear_dens = M_spur / (10*u.deg)
stream_linear_dens, spur_linear_dens

## Select an existing HSC field at comparable Galactic latitude

In [ ]:
hsc = Table.read('/Users/adrian/data/HSC/hsc-stars-gri.fits')

In [ ]:
c = coord.SkyCoord(ra=hsc['ra']*u.deg, dec=hsc['dec']*u.deg)

In [ ]:
gal = c.transform_to(coord.Galactic)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(gal.l.degree[::64],
        gal.b.degree[::64], 
        marker='.', ls='none')
ax.axhline(52)
ax.axhline(-52)
ax.axvline(188)
# ax.set_xlim(0, 360)
ax.set_xlim(73, 78)
ax.set_ylim(-55, -50)

In [ ]:
fr = coord.SkyOffsetFrame(origin=coord.Galactic(l=75*u.deg, b=-52*u.deg))
c_fr = c.transform_to(fr)

# mask = np.sqrt(c_fr.lon.wrap_at(180*u.deg)**2 + c_fr.lat**2) < 1.5*u.deg
mask = np.abs(c_fr.lon.wrap_at(180*u.deg)) < 2.25*u.deg
mask &= np.abs(c_fr.lat) < 0.75*u.deg
print(mask.sum())

fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree,
        c_fr.lat.degree,
        marker='.', ls='none')
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask],
        c_fr.lat.degree[mask],
        marker='.', ls='none')
ax.set_xlim(-2.25, 2.25)
ax.set_ylim(-1.5, 1.5)

### Over-plot GD-1 isochrone

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)

m_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value
ax.plot(iso['ps1_g']-iso['ps1_i'], m_g,
        marker='', lw=3, color='tab:orange', alpha=0.8)

ax.set_xlim(-1, 3)
ax.set_ylim(26., 19)
ax.set_xlabel('$g-i$')
ax.set_ylabel('$g$')
ax.axhline(24, color='#666666', zorder=-100)
fig.set_facecolor('w')

### Use isochrone to estimate number of stars to sample (following mass function) in the field:

In [ ]:
lf_mask = (m_g > 20.) & (m_g < 26.2)
lf_iso = iso[lf_mask]

In [ ]:
M_stream = stream_linear_dens * 3*u.deg # mass in GD-1 stream stars in 1 field
M_spur = spur_linear_dens * 3*u.deg # mass in GD-1 spur stars in 1 field
(((lf_iso['int_IMF'][1:] - lf_iso['int_IMF'][:-1]) / u.Msun * M_stream).sum(),
 ((lf_iso['int_IMF'][1:] - lf_iso['int_IMF'][:-1]) / u.Msun * M_spur).sum())

In [ ]:
from scipy.interpolate import interp1d
interp_f = interp1d(lf_iso['int_IMF'], lf_iso['M_act'])
stream_masses = interp_f(np.random.uniform(lf_iso['int_IMF'].min(), lf_iso['int_IMF'].max(), 1000))
spur_masses = interp_f(np.random.uniform(lf_iso['int_IMF'].min(), lf_iso['int_IMF'].max(), 580))
stream_masses.sum(), spur_masses.sum()

In [ ]:
mg_f = interp1d(lf_iso['M_act'], m_g[lf_mask])
gi_f = interp1d(m_g[lf_mask], (iso['ps1_g']-iso['ps1_i'])[lf_mask])

stream_sim_mg = mg_f(stream_masses)
stream_sim_gi = gi_f(stream_sim_mg)

spur_sim_mg = mg_f(spur_masses)
spur_sim_gi = gi_f(spur_sim_mg)

In [ ]:
plt.plot(hsc['g_psfflux_mag'][mask],
        np.sqrt(hsc['g_psfflux_magsigma'][mask]**2+
                hsc['i_psfflux_magsigma'][mask]**2),
        marker='.', ls='none', alpha=0.2)
plt.xlim(20, 26)
plt.ylim(1e-3, 1e0)
plt.yscale('log')

stream_sim_gi_err = 10**((-0.8 - -3) / (26 - 20) * (stream_sim_mg - 20) + -3)
plt.plot(stream_sim_mg, stream_sim_gi_err, marker='.', ls='none', color='k')

spur_sim_gi_err = 10**((-0.8 - -3) / (26 - 20) * (spur_sim_mg - 20) + -3)
plt.plot(spur_sim_mg, spur_sim_gi_err, marker='.', ls='none', color='k')

stream_sim_gi = np.random.normal(stream_sim_gi, np.sqrt(stream_sim_gi_err**2 + 0.025**2), 
                                 size=len(stream_sim_mg))
spur_sim_gi = np.random.normal(spur_sim_gi, np.sqrt(spur_sim_gi_err**2 + 0.025**2), 
                               size=len(spur_sim_mg))

plt.axvline(24)
plt.axhline(0.15)

In [ ]:
i_g = iso['ps1_g'] + coord.Distance(8*u.kpc).distmod.value
i_gi = iso['ps1_g'] - iso['ps1_i']

i_left = i_gi - 0.4*(i_g/27)**5
i_right = i_gi + 0.4*(i_g/27)**5

poly = np.hstack([np.array([i_left, i_g - 0.5/2.]), 
                  np.array([i_right[::-1], i_g[::-1] + 0.5/2.])]).T
ind = (poly[:,1] < 26) & (poly[:,1] > 20)
# ind = (poly[:,1] < 24) & (poly[:,1] > 20.)
poly_main = poly[ind]
cmd_path_main = mpl.path.Path(poly_main)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 6),
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)

ax.plot(stream_sim_gi, 
        stream_sim_mg,
        marker='.', ls='none', color='k',
        alpha=0.5)


ax = axes[1]
ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)

ax.plot(spur_sim_gi, 
        spur_sim_mg,
        marker='.', ls='none', color='k',
        alpha=0.5)


ax = axes[2]
ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask],
        marker='.', ls='none', color='k',
        alpha=0.5)


ax = axes[3]
ax.plot(stream_sim_gi, 
        stream_sim_mg,
        marker='.', ls='none', color='k',
        alpha=0.5)


ax.set_xlim(-1, 3)
ax.set_ylim(26., 19)
axes[0].set_xlabel('$g-i$')
axes[1].set_xlabel('$g-i$')
axes[0].set_ylabel('$g$')
# ax.axhline(24, color='#666666', zorder=-100)
fig.set_facecolor('w')

axes[0].set_title('stream field')
axes[1].set_title('spur field')
axes[2].set_title('HSC background')
axes[3].set_title('GD-1 CMD model')

for ax in axes:
    ax.plot(poly_main[:, 0], poly_main[:, 1], 
            color='tab:red')

fig.tight_layout()

In [ ]:
bg_colmag = np.vstack((hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
                       hsc['g_psfflux_mag'][mask])).T
bg_iso_mask = cmd_path_main.contains_points(bg_colmag)

fg_stream_colmag = np.vstack((stream_sim_gi, stream_sim_mg)).T
fg_stream_iso_mask = cmd_path_main.contains_points(fg_stream_colmag)

fg_spur_colmag = np.vstack((spur_sim_gi, spur_sim_mg)).T
fg_spur_iso_mask = cmd_path_main.contains_points(fg_spur_colmag)

In [ ]:
bg_iso_mask.sum(), fg_stream_iso_mask.sum(), fg_spur_iso_mask.sum()

In [ ]:
g_bins = np.linspace(20, 26, 16)

n_bg, _ = np.histogram(hsc['g_psfflux_mag'][mask][bg_iso_mask], bins=g_bins)
n_bg = n_bg / 1.5 # to account for larger area

n_stream, _ = np.histogram(stream_sim_mg[fg_stream_iso_mask], bins=g_bins)
n_spur, _ = np.histogram(spur_sim_mg[fg_spur_iso_mask], bins=g_bins)

_binc = 0.5*(g_bins[:-1]+g_bins[1:])
# plt.plot(_binc, n_bg, marker='', drawstyle='steps-mid')
# plt.plot(_binc, n_stream, marker='', drawstyle='steps-mid')
# plt.plot(_binc, n_spur, marker='', drawstyle='steps-mid')

plt.plot(_binc, np.cumsum(n_bg), 
         marker='', drawstyle='steps-mid', label='background', 
         color='k', zorder=-10)
plt.plot(_binc, np.cumsum(n_stream), 
         marker='', drawstyle='steps-mid', label='stream field')
plt.plot(_binc, np.cumsum(n_spur), 
         marker='', drawstyle='steps-mid', label='spur field')
plt.legend(loc='best', fontsize=14)
plt.yscale('log')
plt.ylabel(r'cumulative $N$')
plt.xlabel('$g$ [mag]')

In [ ]:
_binc = 0.5*(g_bins[:-1]+g_bins[1:])
np.save('g_nbg.npy', np.vstack((_binc, n_bg)))
np.save('g_nstream.npy', np.vstack((_binc, n_stream)))

fig, ax = plt.subplots(1, 1, figsize=(6, 5))

ax.plot(_binc, np.cumsum(n_stream) / np.sqrt(np.cumsum(n_bg)), 
        marker='',  label='stream field',
        lw=2.)

ax.plot(_binc, np.cumsum(n_spur) / np.sqrt(np.cumsum(n_bg)), 
        marker='', label='spur field',
        lw=2)

# ax.plot(_binc, np.cumsum(n_stream / np.sqrt(n_stream)), 
#         marker='',  label='stream field',
#         lw=2.)

# ax.plot(_binc, np.cumsum(n_spur / np.sqrt(n_bg)), 
#         marker='', label='spur field',
#         lw=2)

ax.legend(loc='best', fontsize=18)

ax.set_xlabel('$g$ [mag]')
ax.set_ylabel(r'cumulative S/N')

ax.set_xticks(np.arange(20, 26+1e-3, 1))
ax.set_xlim(20, 26)
ax.set_ylim(1, 34)

ax.axvline(23., alpha=0.4, 
           color='k', linestyle='--', zorder=-10)
ax.axvline(25.5, alpha=0.6, 
           color='k', zorder=-10)

fig.set_facecolor('w')

fig.tight_layout()

In [ ]:
for f in [1.01, 1.2, 1.3, 2, 4]:
    err_f = np.sqrt(np.cumsum(n_bg)) / (f**2 * np.cumsum(n_stream)) * np.sqrt(2*(1+f**2) + f*(1+f)*np.cumsum(n_stream)/np.cumsum(n_bg))
#     plt.plot(_binc, err_f) # / f)
    plt.plot(_binc, (f-1) / err_f)

plt.axhline(3.)
plt.yscale('log')

---

In [ ]:
plt.figure()
_, bins, _ = plt.hist(main.phi2, bins='auto', density=True, alpha=0.5)
plt.hist(np.random.normal(0, 0.17, size=1000), density=True, alpha=0.5)

stream_sim_phi1 = np.random.uniform(-1.5, 1.5, size=len(stream_sim_gi))
stream_sim_phi2 = np.random.normal(0, 0.17, size=len(stream_sim_gi))

spur_sim_phi1 = np.random.uniform(-1.5, 1.5, size=len(spur_sim_gi))
spur_sim_phi2 = np.random.normal(0, 0.5, size=len(spur_sim_gi))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5), 
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask][bg_iso_mask],
        c_fr.lat.degree[mask][bg_iso_mask],
        marker='.', ls='none', color='k')

ax.plot(stream_sim_phi1[fg_stream_iso_mask], 
        stream_sim_phi2[fg_stream_iso_mask],
        marker='.', ls='none', color='k')

ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)

ax = axes[1]
ax.plot(c_fr.lon.wrap_at(180*u.deg).degree[mask][bg_iso_mask],
        c_fr.lat.degree[mask][bg_iso_mask],
        marker='.', ls='none', color='k')

ax.plot(spur_sim_phi1[fg_spur_iso_mask], spur_sim_phi2[fg_spur_iso_mask],
        marker='.', ls='none', color='k')

axes[0].set_xlabel('lon [deg]')
axes[0].set_ylabel('lat [deg]')
axes[0].set_title('stream field')
axes[1].set_title('spur field')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))

ax.plot(hsc['g_psfflux_mag'][mask] - hsc['i_psfflux_mag'][mask],
        hsc['g_psfflux_mag'][mask] - hsc['z_psfflux_mag'][mask],
        marker=',', ls='none', color='k',
        alpha=0.5)

ax.set_xlim(-1, 3)
ax.set_ylim(-3, 4)
ax.set_xlabel('$g-i$')
ax.set_ylabel('$g-z$')
fig.set_facecolor('w')